In [65]:
import xarray as xr
import proplot as pplt
import pandas as pd
from AOSCMcoupling.files import OIFSPreprocessor, NEMOPreprocessor
import numpy as np
import warnings
from pathlib import Path

In [66]:
version_marker = "ECE3, OpenIFS 40r1"
max_iters = 40

In [67]:
def load_iterates(
    file_name: str, preprocess: callable, max_iters: int, dir: Path,
) -> xr.Dataset:
    assert(Path(dir).is_dir())
    swr_dim = xr.DataArray(np.arange(max_iters) + 1, dims="swr_iterate")
    iterates = [
        xr.open_mfdataset(f"{dir}/iter_{iter}/{file_name}", preprocess=preprocess)
        for iter in range(1, max_iters + 1)
    ]
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        iterates = xr.concat(iterates, swr_dim)
    return iterates

In [68]:
start_date_warm = pd.Timestamp("2020-07-03 00:00")
oifs_preprocessor = OIFSPreprocessor(start_date_warm)
nemo_preprocessor = NEMOPreprocessor(start_date_warm)

oifs_progvars = []

dates = ["2014-07-03_00", "2014-07-12_18", "2014-07-25_12"]
dates_43 = [
    pd.Timestamp("2014-07-19 00:00"),
    pd.Timestamp("2014-07-07 06:00"),
    pd.Timestamp("2014-07-10 18:00"),
    pd.Timestamp("2014-07-24 18:00"),
    pd.Timestamp("2014-07-07 18:00"),
    pd.Timestamp("2014-07-28 18:00"),
    pd.Timestamp("2014-07-17 06:00"),
    pd.Timestamp("2014-07-26 12:00"),
    pd.Timestamp("2014-07-23 06:00"),
    pd.Timestamp("2014-07-11 18:00"),
    pd.Timestamp("2014-07-18 06:00"),
]
dates_4 = [
    pd.Timestamp("2014-07-11 12:00"),
    pd.Timestamp("2014-07-02 12:00"),
    pd.Timestamp("2014-07-24 18:00"),
    pd.Timestamp("2014-07-23 12:00"),
    pd.Timestamp("2014-07-18 06:00"),
    pd.Timestamp("2014-07-14 18:00"),
    pd.Timestamp("2014-07-23 00:00"),
]
dates = [f"{date.date()}_{date.time().hour:02}" for date in dates_4]
dirs = ["output/ensemble_output_nc/" + date for date in dates]
for dir in dirs:
    oifs_progvar = load_iterates(
        "progvar.nc", oifs_preprocessor.preprocess, max_iters, dir
    )
    oifs_progvars.append(oifs_progvar)

In [ ]:
oifs_progvars[0]

In [ ]:
fig, axs = pplt.subplots(ncols=len(dates), sharey=1)

for index, date in enumerate(dates):
    ax = axs[index]
    data = oifs_progvars[index].t.isel(time=-1, nlev=-1) - 273.15
    data = data.sel(swr_iterate=range(20, 41))
    ax.plot(data, color="k", marker=".")
    ax.format(
        ylabel="Temperature [°C]",
        xlabel="Iteration",
        xlim=[20, 40],
        title=date,
    )

axs.format(
    abc="a)",
    abcloc='ul',
)
fig.savefig("nwp_oscillations_ece4.pdf")

In [96]:
dates_3 = np.array([
    pd.Timestamp("2014-07-03 00:00:00"),
    pd.Timestamp("2014-07-12 18:00:00"),
    pd.Timestamp("2014-07-25 12:00:00"),
])
dates_43 = np.array([
    pd.Timestamp("2014-07-19 00:00"),
    pd.Timestamp("2014-07-07 06:00"),
    pd.Timestamp("2014-07-10 18:00"),
    pd.Timestamp("2014-07-24 18:00"),
    pd.Timestamp("2014-07-07 18:00"),
    pd.Timestamp("2014-07-28 18:00"),
    pd.Timestamp("2014-07-17 06:00"),
    pd.Timestamp("2014-07-26 12:00"),
    pd.Timestamp("2014-07-23 06:00"),
    pd.Timestamp("2014-07-11 18:00"),
    pd.Timestamp("2014-07-18 06:00"),
])
dates_4 = np.array([
    pd.Timestamp("2014-07-11 12:00"),
    pd.Timestamp("2014-07-02 12:00"),
    pd.Timestamp("2014-07-24 18:00"),
    pd.Timestamp("2014-07-23 12:00"),
    pd.Timestamp("2014-07-18 06:00"),
    pd.Timestamp("2014-07-14 18:00"),
    pd.Timestamp("2014-07-23 00:00"),
])
all_dates = np.array(pd.date_range(pd.Timestamp("2014-07-01 00:00"), pd.Timestamp("2014-07-28 18:00"), freq=pd.Timedelta(6, "h")), dtype=pd.Timestamp)


bdates_4 = np.zeros(all_dates.shape)
for index, date in enumerate(all_dates):
    if date in dates_4:
        bdates_4[index] = 1

bdates_3 = np.zeros(all_dates.shape)
for index, date in enumerate(all_dates):
    if date in dates_3:
        bdates_3[index] = 1

bdates_43 = np.zeros(all_dates.shape)
for index, date in enumerate(all_dates):
    if date in dates_43:
        bdates_43[index] = 1

In [ ]:
fig, ax = pplt.subplots(height="10em", width="80em")
ax.pcolormesh(np.array([bdates_3, bdates_43, bdates_4]))
ax.format(yticks=[0,1,2],yticklabels=["ECE3", "ECE43", "ECE4"],ytickminor=False)